In [ ]:
import geopandas as gpd 
import osmnx as ox 
import pandas as pd 
import matplotlib.pyplot as plt 
from matplotlib.lines import Line2D 
from matplotlib.animation import FuncAnimation 
import city2graph as c2g

In [ ]:
# Download the street network for Waterloo, ON, Canada
G = ox.graph_from_place(
    "Kitchener, Canada",
    network_type="all",
)

In [ ]:
street_primary_nodes, street_primary_edges = c2g.nx_to_gdf(G)
street_primary_nodes = street_primary_nodes.to_crs(epsg=27700)
street_primary_edges = street_primary_edges.to_crs(epsg=27700)

amenity_tags = ["cafe", "restaurant", "pub", "bar", "museum", "theatre", "cinema"]
# amenity_tags = ["theatre", "cinema"]
amenity_candidates = ox.features_from_place(
    "Kitchener, Canada",
    tags={"amenity": amenity_tags},
).to_crs(epsg=27700)

In [ ]:
# Collapse complex Amenity geometries to points within the projected CRS 
amenities = ( 
    amenity_candidates[["name", "amenity", "geometry"]] 
    .copy() 
    .explode(index_parts=False) 
    .dropna(subset=["geometry"]) 
    ) 

non_point_mask = ~amenities.geometry.geom_type.isin(["Point"]) 
amenities.loc[non_point_mask, "geometry"] = amenities.loc[non_point_mask, "geometry"].centroid 
amenities = amenities.set_geometry("geometry") 
amenities["name"] = amenities["name"].fillna(amenities["amenity"].str.title()) 
amenities = amenities[~amenities.geometry.is_empty] 
amenities = amenities.drop_duplicates(subset="geometry").reset_index(drop=True)

In [ ]:
# Display street network data 
print("Street Primary Nodes:") 
display(street_primary_nodes.head(3)) 
print("\nStreet Primary Edges:") 
display(street_primary_edges.head(3)) 
print("\nAmenities:") 
display(amenities.head(3))

In [ ]:
c2g.plot_graph(nodes=street_primary_nodes, edges=street_primary_edges)

# Create a dual graph to convert intersections to edges and streets to nodes

In [ ]:
street_dual_nodes, street_dual_edges = c2g.dual_graph((street_primary_nodes, street_primary_edges))

street_dual_nodes.geometry = street_dual_nodes.geometry.centroid

In [ ]:
# Display dual graph data
print("Street Dual Nodes (Segments):")
display(street_dual_nodes.head(3))
print("\nStreet Dual Edges (Connections):")
display(street_dual_edges.head(3))

In [ ]:
c2g.plot_graph(nodes=street_dual_nodes, 
               edges=street_dual_edges)

# Bridge amenities (primary nodes) to nearby streets (dual nodes)

In [ ]:
nodes_dict = { "amenity": amenities, "segment": street_dual_nodes } 
edges_dict = { ("segment", "connects_to", "segment"): street_dual_edges } 

# Connect amenities to segments # bridge_nodes returns the node dictionary and a new dictionary of proximity edges 
_, bridged_edges = c2g.bridge_nodes( 
    nodes_dict=nodes_dict, 
    proximity_method="knn", 
    source_node_types=["amenity"], 
    target_node_types=["segment"], 
    k=1 # Connect to the single nearest segment 
    ) # Add the street network edges to the edges dictionary edges_dict.update(bridged_edges)

In [ ]:
nodes_dict = {
    "amenity": amenities,
    "segment": street_dual_nodes
}

edges_dict = {
    ("segment", "connects_to", "segment"): street_dual_edges
}

# Connect amenities to segments
# bridge_nodes returns the node dictionary and a new dictionary of proximity edges
_, bridged_edges = c2g.bridge_nodes(
    nodes_dict=nodes_dict,
    proximity_method="knn",
    source_node_types=["amenity"],
    target_node_types=["segment"],
    k=1  # Connect to the single nearest segment
)

# Add the street network edges to the edges dictionary
edges_dict.update(bridged_edges)

In [ ]:
bridged_edges.keys()

In [ ]:
# Define sequence for 1 to 10 street hops 
sequence = [] 
hops = 3 
# Start: Amenity -> Segment 
sequence = [("amenity", "is_nearby", "segment")] 

# Middle: Segment -> Segment (i times) 
for _ in range(hops): 
    sequence.append(("segment", "connects_to", "segment")) 

# End: Segment -> Amenity 
sequence.append(("segment", "is_nearby", "amenity")) 

print(sequence)

In [ ]:
# Materialize the metapath edges 
result_nodes, result_edges = c2g.add_metapaths(nodes=nodes_dict, 
                                               edges=edges_dict, 
                                               sequence=sequence, 
                                               new_relation_name="is_3_hop_nearby")

In [ ]:
# Visualize the graph with metapaths 
# # We want to highlight the metapath connections 
# # Plot 
c2g.plot_graph( 
    nodes=result_nodes, 
    edges=result_edges, 
    node_color={ "amenity": "red", "segment": "gray" }, 
    node_zorder={ "amenity": 3, "segment": 1 }, 
    node_alpha={ "amenity": 1.0, "segment": 0.5 }, 
    markersize={ "amenity": 50, "segment": 5 }, 
    edge_color={ ("segment", "connects_to", "segment"): "gray", ("amenity", "is_nearby", "segment"): "gray", ("amenity", "is_3_hop_nearby", "amenity"): "cyan", }, 
    edge_zorder={ ("segment", "connects_to", "segment"): 3, ("amenity", "is_nearby", "segment"): 1 }, 
    edge_linewidth={ ("segment", "connects_to", "segment"): 0.5, ("amenity", "is_nearby", "segment"): 0.5, ("amenity", "is_3_hop_nearby", "amenity"): 2.0, }, 
    bgcolor="black", legend_position="lower right", subplots=False )